# ISSIA Processing with Progress Tracking

This notebook shows how to process ATCOR-4 files with detailed progress information.

In [ ]:
# Install required packages if needed
# !pip install tqdm dask[array] rasterio numpy scipy matplotlib xarray

In [1]:
import numpy as np
from pathlib import Path
from issia_notebook import ISSIAProcessorNotebook, process_with_monitoring
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

import importlib
import issia
importlib.reload(issia)

import spectral.io.envi as envi


# from dask.distributed import Client
# client = Client(n_workers=4, threads_per_worker=2)

✓ Imports successful


## Step 1: Initialize Processor

Set `verbose=True` to see detailed progress at each step.

In [2]:
# Define instrument parameters

# data_dir = Path('/Volumes/aco-uvic/2022_Acquisitions/02_Processed/22_4012_07_PlaceGlacier/03_Hyper/02_Working/OUTPUT/subsets/')
# flight_line = '22_4012_07_2022-08-07_19-54-01-rect_img'

# hdr = envi.open(f"{data_dir}/{flight_line}_atm.hdr")
# wvl = [float(w) for w in hdr.metadata['wavelength']]

# wavelengths = np.array([float(w) for w in hdr.metadata['wavelength']])

wavelengths = np.load('wvl.npy')

#wavelengths = np.linspace(380, 2500, 451)  # AisaFENIX example

# LUT dimensions (must match your generated LUTs)
grain_radii = np.arange(30, 5001, 30),  # Matches MATLAB: 30:30:10000
illumination_angles = np.arange(0, 86, 5)
viewing_angles = np.arange(0, 86, 5)
relative_azimuths = np.arange(0, 361, 10)


# Initialize processor with verbose mode
processor = ISSIAProcessorNotebook(
    wavelengths=wavelengths,
    grain_radii=grain_radii,
    illumination_angles=illumination_angles,
    viewing_angles=viewing_angles,
    relative_azimuths=relative_azimuths,
    coord_ref_sys_code=32610,  # Adjust for your area
    chunk_size=(512, 512),      # Adjust based on available RAM
    verbose=True                # Enable progress messages
)

print("\n✓ Processor initialized")
print(f"  Wavelengths: {len(wavelengths)}")
print(f"  Grain sizes: {len(grain_radii)}")
print(f"  Chunk size: {processor.chunk_size}")


✓ Processor initialized
  Wavelengths: 451
  Grain sizes: 1
  Chunk size: (512, 512)


## Step 2: Load Lookup Tables

This will show progress as each LUT is loaded.

In [3]:
lut_dir = Path("lookup_tables")

processor.load_lookup_tables(
    sbd_lut_path=lut_dir / "sbd_lut.npy",
    anisotropy_lut_path=lut_dir / "anisotropy_lut.npy",
    albedo_lut_path=lut_dir / "albedo_lut.npy"
)

print("\n✓ All lookup tables loaded successfully")

Loading lookup tables...
Lookup tables loaded successfully

✓ All lookup tables loaded successfully


## Step 3: Process Flight Line

Now process with detailed progress at each step:

In [ ]:

# Set paths

#data_dir=Path("testing/")
#data_dir=Path("/Volumes/aco-uvic/2021_Acquisitions/02_Processed/21_4012_06_Place_CryoMB/03_Hyper/02_Working/OUTPUT/subsets/")
data_dir = Path('/Volumes/aco-uvic/2022_Acquisitions/02_Processed/22_4012_07_PlaceGlacier/03_Hyper/02_Working/OUTPUT/subsets/')
flight_line = '22_4012_07_2022-08-07_19-54-01-rect_img'

output_dir = Path("issia_results")



# Test on just 500x500 pixels (top-left corner)
output_files = processor.process_flight_line(
    data_dir=data_dir,
    flight_line=flight_line,
    
    subset=(500, 2500, 500, 2500),

    output_dir=output_dir
)





PROCESSING FLIGHT LINE: 22_4012_07_2022-08-07_19-54-01-rect_img
SUBSET MODE: rows 500-2500, cols 500-2500

[13:41:25] INFO: Reading ATCOR files for flight line: 22_4012_07_2022-08-07_19-54-01-rect_img
[13:41:25] INFO: Checking for required files...
[13:41:25] INFO:   ✓ Found 22_4012_07_2022-08-07_19-54-01-rect_img.inn (0.0 MB)
[13:41:25] INFO:   ✓ Found 22_4012_07_2022-08-07_19-54-01-rect_img_atm.dat (4454.3 MB)
[13:41:25] INFO:   ✓ Found 22_4012_07_2022-08-07_19-54-01-rect_img_eglo.dat (4454.3 MB)
[13:41:25] INFO:   ✓ Found 22_4012_07_2022-08-07_19-54-01-rect_img_slp.dat (19.7 MB)
[13:41:25] INFO:   ✓ Found 22_4012_07_2022-08-07_19-54-01-rect_img_asp.dat (19.7 MB)
Reading subset window: rows 500-2500, cols 500-2500


Loading 5 files...
  [1/5] Loading Reflectance...

## Step 4: Visualize Results

Quick visualization of the outputs:

In [ ]:
import matplotlib.pyplot as plt
import rasterio

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Load and plot grain size
with rasterio.open(output_files['grain_size']) as src:
    grain_size = src.read(1)

im1 = axes[0].imshow(grain_size, cmap='Greens', vmin=50, vmax=100)
axes[0].set_title('Grain Size (μm)', fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# Load and plot broadband albedo
with rasterio.open(output_files['broadband_albedo']) as src:
    albedo = src.read(1)

im2 = axes[1].imshow(albedo, cmap='gray', vmin=0.3, vmax=0.95)
axes[1].set_title('Broadband Albedo', fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046)

# Load and plot radiative forcing
with rasterio.open(output_files['radiative_forcing']) as src:
    rf = src.read(1)

im3 = axes[2].imshow(rf, cmap='RdPu', vmin=0, vmax=100)
axes[2].set_title('Radiative Forcing (W/m²)', fontweight='bold')
axes[2].axis('off')
plt.colorbar(im3, ax=axes[2], fraction=0.046)

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete")